In [ ]:
import pymysql
import pandas as pd

In [ ]:
import pymysql
import pandas as pd

# Correct DataFrame (no duplicate column names)
df = pd.DataFrame({
    'sepal_length': [5.1, 4.9, 4.7],
    'sepal_width': [3.5, 3.0, 3.2],
    'petal_length': [1.4, 1.4, 1.3],
    'petal_width': [0.2, 0.2, 0.2],
    'species': ['setosa', 'setosa', 'setosa']
})

try:
  # Connect to MariaDB
  conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    port=3306,
    password='NewStrongPasswordHere',
    database='scrapped_data_database_1'
  )
  
  with conn.cursor() as cursor:
    # Create table
    cursor.execute("""
      CREATE TABLE IF NOT EXISTS Iris (
        sepal_length FLOAT,
        sepal_width FLOAT,
        petal_length FLOAT,
        petal_width FLOAT,
        species VARCHAR(20)
      )
    """)

    # Insert data
    insert_query = """
      INSERT INTO Iris (sepal_length, sepal_width, petal_length, petal_width, species)
      VALUES (%s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_query, df.values.tolist())

  conn.commit()
  print("✅ Data inserted successfully.")

except Exception as e:
  print("❌ Error:", e)

finally:
  if conn:
    conn.close()
    print("🔒 Connection closed.")


In [ ]:
# Retrieve the data back to verify
import pymysql
import pandas as pd

# Connect to MariaDB
conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    password='NewStrongPasswordHere',
    database='scrapped_data_database_1'
)

# Use pandas to read the SQL table into a DataFrame
df_retrieved = pd.read_sql("SELECT * FROM Iris", conn)

conn.close()

# Show the DataFrame
df_retrieved.head()


In [1]:
import pandas as pd
import pymysql

def generate_create_table_sql(df, table_name="final_dataset"):
    dtype_mapping = {
        'int64': 'INT',
        'float64': 'DOUBLE',
        'object': 'VARCHAR(255)',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'DATETIME',
    }

    columns = []
    for col, dtype in df.dtypes.items():
        col_type = dtype_mapping.get(str(dtype), 'VARCHAR(255)')
        if col == df.columns[0] and col_type in ['INT', 'BIGINT']:
            columns.append(f"{col} {col_type} PRIMARY KEY")
        else:
            columns.append(f"{col} {col_type}")
    
    columns_sql = ",\n    ".join(columns)
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n    {columns_sql}\n);"
    return create_table_sql

def main():
    # Load the DataFrame
    df = pd.read_csv("./transformed_data_final.csv")
    table_name = "final_dataset"

    # Generate SQL to create the table
    create_table_sql = generate_create_table_sql(df, table_name)

    # Database credentials
    db_config = {
        "host": "localhost",
        "user": "birat-gautam",
        "password": "NewStrongPasswordHere",
        "database": "scrapped_data_database_1",
        "port": 3306
    }

    try:
        # Connect to the database
        conn = pymysql.connect(**db_config)
        cursor = conn.cursor()

        # Create the table
        cursor.execute(create_table_sql)
        print(f"✅ Table `{table_name}` created or already exists.")

        # Prepare insert query
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_query = f"""
            INSERT INTO {table_name} ({', '.join(df.columns)})
            VALUES ({placeholders})
        """

        # Insert data
        cursor.executemany(insert_query, df.values.tolist())
        conn.commit()
        print(f"✅ Inserted {len(df)} rows into `{table_name}`")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()
            print("🔒 Connection closed.")

if __name__ == "__main__":
    main()


✅ Table `final_dataset` created or already exists.
✅ Inserted 7260 rows into `final_dataset`
🔒 Connection closed.


## **Redis Connection**


In [2]:
import redis
import json
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Connect to Redis
redis_client = redis.Redis(host='localhost', port=6379, decode_responses=False)

try:
  # Test Redis connection
  redis_client.ping()
  print("✅ Connected to Redis successfully")
  
  # Connect to MariaDB to fetch data
  conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    password='NewStrongPasswordHere',
    database='scrapped_data_database_1',
    port=3306
  )
  
  # Fetch data from MariaDB
  df_from_db = pd.read_sql("SELECT * FROM final_dataset", conn)
  print(f"✅ Fetched {len(df_from_db)} rows from MariaDB")
  
  # Convert DataFrame to PyArrow Table and then to Parquet bytes
  table = pa.Table.from_pandas(df_from_db)
  buffer = BytesIO()
  pq.write_table(table, buffer)
  parquet_bytes = buffer.getvalue()
  
  # Store Parquet data in Redis
  redis_client.set('final_dataset_parquet', parquet_bytes)
  print("✅ Data stored in Redis as Parquet with key 'final_dataset_parquet'")

  # Retrieve data from Redis and convert back to DataFrame
  retrieved_bytes = redis_client.get('final_dataset_parquet')
  retrieved_buffer = BytesIO(retrieved_bytes)
  retrieved_table = pq.read_table(retrieved_buffer)
  retrieved_df = retrieved_table.to_pandas()
  
  print("✅ Data retrieved from Redis:")
  print(retrieved_df.head())
  print(f"Shape: {retrieved_df.shape}")
  
except redis.ConnectionError:
  print("❌ Failed to connect to Redis")
except Exception as e:
  print(f"❌ Error: {e}")
finally:
  if 'conn' in locals():
    conn.close()
    print("🔒 MariaDB connection closed")

✅ Connected to Redis successfully
✅ Fetched 7260 rows from MariaDB
✅ Data stored in Redis as Parquet with key 'final_dataset_parquet'
✅ Data retrieved from Redis:
  FTR  HTP  ATP HM1 HM2 HM3 AM1 AM2 AM3  HTGD  ATGD  DiffFormPts
0   H  0.0  0.0   M   M   M   M   M   M   0.0   0.0          0.0
1   H  0.0  0.0   M   M   M   M   M   M   0.0   0.0          0.0
2  NH  0.0  0.0   M   M   M   M   M   M   0.0   0.0          0.0
3  NH  0.0  0.0   M   M   M   M   M   M   0.0   0.0          0.0
4   H  0.0  0.0   M   M   M   M   M   M   0.0   0.0          0.0
Shape: (7260, 12)
🔒 MariaDB connection closed


/tmp/ipykernel_9770/985674556.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_from_db = pd.read_sql("SELECT * FROM final_dataset", conn)
